In [1]:
import pandas as pd
import MeCab
import fitz
import re
import sys
import itertools
from collections import Counter


In [2]:

#text = "管理籍"
#text="そして、このしくみの対象となる当社グループ会社（国内）は、みずから生み出しまたは譲り受けた特許等を、PHDへの法的名義集約後も管理し（「管理籍」の保持）、当該特許等の費用を負担し、収益を収受します。"
#text="現在のソニーでは、Purposeの推進に向け、これら価値創造のドライバーに対する経営資源の重点配分や、ドライバーのさらなる融合に注力しています。事業間連携は定着しつつあり、コンテンツIPのグループ内での展開によるIP価値向上や、事業を横断したテクノロジーの活用プロジェクトが増加しています。"
text="環境委員会	・カーボンニュートラル（Scope1、2）達成に向けた2024年度以降の削減計画・移行リスクと物理的リスクにかかるシナリオ分析の高度化について・地域のカーボンニュートラル実現に向けた各種施策（地域のお客さまへのソリューション支援・啓発策、地方公共団体等との連携を含む）・地域の取引先等に対する温室効果ガス排出量の算定支援（温室効果ガス排出量算定サービス「しずおかGXサポート」の展開）・2024年度サステナブルファイナンス(環境関連ファイナンスを含む)目標の設定・TNFD Adopterへの登録と今後の方針。\n 気候変動による当グループへの影響を把握するため、シナリオ分析（気候変動に関するリスクが与信ポートフォリオに与える影響を把握）を実施しています。\n使用したシナリオ	・IEA・50年実質ゼロシナリオ（NZE2050）・IEA・ETP2017Beyond2℃シナリオ・NGFS Net Zero 2050。\n分析方法	シナリオに基づき炭素税等の予測データを使用して、2050年までの損益財政状態の変化を予想し、与信費用の変化を分析。\m使用したシナリオ	IPCC第6次報告書におけるRCP8.5（4℃シナリオ）。"

#text = "貴社の記者が汽車で帰社した。"
#text = "雨がしとしと降っている。"

In [3]:
#共通パスを定義
path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
path2 = " -u /work_dir/userdic/user.dic"

In [4]:
def talknizer_bare(text,path1):
    
#    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    #path1 = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path1)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t") for l in parsed_lines]#原形を取得

    return token_list

def token_bare(text,path1):
    
    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
#    path1 = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path1)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    return token_list


def token_with_userdic(text,path1,path2):
    
    #path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    #path2 = " -u /work_dir/userdic/user.dic"
    mecab = MeCab.Tagger(path1+path2)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    return token_list

def token_noun(text,path1):
    
    #path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
#    path1 = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path1)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    #--------------------------------------------
    #品詞区分による絞り込み：Chasenの品詞区分を個別に指定する場合
    #品詞区分の取得（リスト形式）
    pos = [l.split('\t')[3] for l in parsed_lines]

    #抽出する品詞区分の定義（完全一致で指定）
    target_pos = ['名詞-一般',
                  '名詞-固有名詞-一般',
                  '名詞-固有名詞-人名-一般',
                  '名詞-固有名詞-人名-姓',
                  '名詞-固有名詞-人名-名',
                  '名詞-固有名詞-組織',
                  '名詞-固有名詞-地域-一般',
                  '名詞-固有名詞-地域-国',
                  '名詞-代名詞-一般',
                  '名詞-代名詞-縮約',
                  '名詞-副詞可能',
                  '名詞-サ変接続',
                  '名詞-形容動詞語幹',
                  '名詞-ナイ形容詞語幹'
                 ]
    #--------------------------------------------
    #形態素と品詞区分のリストをペアにしてタプルリスト化、該当する品詞区分の形態素のみリストに出力
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    return token_list

def token_noun_with_userdic(text,path1,path2):
    
    #path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    #path2 = " -u /work_dir/userdic/user.dic"
    mecab = MeCab.Tagger(path1+path2)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    #--------------------------------------------
    #品詞区分による絞り込み：Chasenの品詞区分を個別に指定する場合
    #品詞区分の取得（リスト形式）
    pos = [l.split('\t')[3] for l in parsed_lines]

    #抽出する品詞区分の定義（完全一致で指定）
    target_pos = ['名詞-一般',
                  '名詞-固有名詞-一般',
                  '名詞-固有名詞-人名-一般',
                  '名詞-固有名詞-人名-姓',
                  '名詞-固有名詞-人名-名',
                  '名詞-固有名詞-組織',
                  '名詞-固有名詞-地域-一般',
                  '名詞-固有名詞-地域-国',
                  '名詞-代名詞-一般',
                  '名詞-代名詞-縮約',
                  '名詞-副詞可能',
                  '名詞-サ変接続',
                  '名詞-形容動詞語幹',
                  '名詞-ナイ形容詞語幹'
                 ]
    #--------------------------------------------
    #形態素と品詞区分のリストをペアにしてタプルリスト化、該当する品詞区分の形態素のみリストに出力
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    #Neolondの辞書の出力結果を補正
    replacement_dict = {}
    with open("userdic/dic_correction.txt","r") as fc:
        correction_list = fc.read()
    for line in correction_list.split("\n"):
        old, new = line.split("\t")
        replacement_dict[old] = new
    #辞書出力を置換マッピングを使って更新
    token_list = [replacement_dict.get(item, item) for item in token_list]
    return token_list



In [5]:
#集約したテキストを「。」で区切り、形態素解析（共起分析用）
sentences = [token_noun_with_userdic(sentence,path1,path2) for sentence in re.split("。", text)]
#sentences = [token_noun(sentence) for sentence in re.split("。", text)]

In [6]:
print(sentences)

[['環境委員会', 'カーボンニュートラル', 'Scope', '達成', '2024年度', '以降', '削減', '計画', '移行', 'リスク', '物理的', 'リスク', 'シナリオ分析', '高度', '地域', 'カーボンニュートラル', '実現', '各種', '施策', '地域', 'お客様', 'ソリューション', '支援', '啓発', '地方公共団体', '連携', '地域', '取引先', '温室効果ガス', '排出', '算定', '支援', '温室効果ガス', '排出', '算定', 'サービス', 'しず', 'GX', 'サポート', '展開', '2024年度', 'サステナブル', 'ファイナンス', '環境', '関連', 'ファイナンス', '目標', '設定', 'TNFD', 'Adopter', '登録', '今後', '方針'], ['気候変動', 'グループ', '影響', '把握', 'シナリオ分析', '気候変動', 'リスク', '与信', 'ポートフォリオ', '影響', '把握', '実施'], ['使用', 'シナリオ', 'IEA', '50年', '実質', 'シナリオ', 'NZE2050', 'IEA', 'ETP', 'BEYOND', '2℃', 'シナリオ', 'NGFS', 'Net Zero 2050'], ['分析', '方法', 'シナリオ', '炭素税', '予測', 'データ', '使用', '2050年', '損益', '財政状態', '変化', '予想', '与信', '費用', '変化', '分析'], ['m', '使用', 'シナリオ', 'IPCC', '第6次', '報告', 'RCP8.5', '4℃', 'シナリオ'], []]


In [7]:
#各文中の、形態素組み合わせを作る
sentences_combs = [list(itertools.combinations(sentence,2)) for sentence in sentences]
#print(sentences_combs[0])

#組み合わせた2つの形態素の並びをソート
words_combs = [[tuple(sorted(words)) for words in sentence] for sentence in sentences_combs]
#print(words_combs[0][:30])
target_combs = []
for words_comb in words_combs:
    target_combs.extend(words_comb)
ct = Counter(target_combs)
#print(ct)
df = pd.DataFrame([{"1番目" : i[0][0], "2番目": i[0][1], "count":i[1]} for i in ct.most_common()])
#df.head(30)


In [8]:
print(talknizer_bare(text,path1))

[['環境委員会', 'カンキョウイインカイ', '環境委員会', '名詞-固有名詞-一般', '', ''], ['・', '・', '・', '記号-一般', '', ''], ['カーボンニュートラル', 'カーボンニュートラル', 'カーボンニュートラル', '名詞-固有名詞-一般', '', ''], ['（', '（', '（', '記号-括弧開', '', ''], ['Scope', 'スコープ', 'Scope', '名詞-固有名詞-一般', '', ''], ['1', '1', '1', '名詞-数', '', ''], ['、', '、', '、', '記号-読点', '', ''], ['2', '2', '2', '名詞-数', '', ''], ['）', '）', '）', '記号-括弧閉', '', ''], ['達成', 'タッセイ', '達成', '名詞-サ変接続', '', ''], ['に', 'ニ', 'に', '助詞-格助詞-一般', '', ''], ['向け', 'ムケ', '向ける', '動詞-自立', '一段', '連用形'], ['た', 'タ', 'た', '助動詞', '特殊・タ', '基本形'], ['2024年度', 'ニセンニジュウヨンネンド', '2024年度', '名詞-固有名詞-一般', '', ''], ['以降', 'イコウ', '以降', '名詞-副詞可能', '', ''], ['の', 'ノ', 'の', '助詞-連体化', '', ''], ['削減', 'サクゲン', '削減', '名詞-サ変接続', '', ''], ['計画', 'ケイカク', '計画', '名詞-サ変接続', '', ''], ['・', '・', '・', '記号-一般', '', ''], ['移行', 'イコウ', '移行', '名詞-サ変接続', '', ''], ['リスク', 'リスク', 'リスク', '名詞-一般', '', ''], ['と', 'ト', 'と', '助詞-並立助詞', '', ''], ['物理的', 'ブツリテキ', '物理的', '名詞-固有名詞-一般', '', ''], ['リスク', 'リスク', 'リスク', '名詞-一般', '', ''], ['に', 'ニ

In [9]:
print(token_bare(text,path1))

['環境委員会', '・', 'カーボンニュートラル', '（', 'Scope', '1', '、', '2', '）', '達成', 'に', '向ける', 'た', '2024年度', '以降', 'の', '削減', '計画', '・', '移行', 'リスク', 'と', '物理的', 'リスク', 'に', 'かかる', 'シナリオ分析', 'の', '高度', '化', 'について', '・', '地域', 'の', 'カーボンニュートラル', '実現', 'に', '向ける', 'た', '各種', '施策', '（', '地域', 'の', 'お客様', 'へ', 'の', 'ソリューション', '支援', '・', '啓発', '策', '、', '地方公共団体', '等', 'と', 'の', '連携', 'を', '含む', '）', '・', '地域', 'の', '取引先', '等', 'に対する', '温室効果ガス', '排出', '量', 'の', '算定', '支援', '（', '温室効果ガス', '排出', '量', '算定', 'サービス', '「', 'しず', 'お', 'か', 'GX', 'サポート', '」', 'の', '展開', '）', '・', '2024年度', 'Sustainable', 'ファイナンス', '(', '環境', '関連', 'ファイナンス', 'を', '含む', ')', '目標', 'の', '設定', '・', 'TNFD', 'Adopter', 'へ', 'の', '登録', 'と', '今後', 'の', '方針', '。', '気候変動', 'による', '当', 'グループ', 'へ', 'の', '影響', 'を', '把握', 'する', 'ため', '、', 'シナリオ分析', '（', '気候変動', 'に関する', 'リスク', 'が', '与信', 'ポートフォリオ', 'に', '与える', '影響', 'を', '把握', '）', 'を', '実施', 'する', 'て', 'いる', 'ます', '。', '使用', 'する', 'た', 'シナリオ', '・', 'IEA', '・', '50年', '実質', 'ゼロ', 'シナリオ', '（',

In [10]:
print(token_with_userdic(text,path1,path2))

['環境委員会', '・', 'カーボンニュートラル', '（', 'Scope', '1', '、', '2', '）', '達成', 'に', '向ける', 'た', '2024年度', '以降', 'の', '削減', '計画', '・', '移行', 'リスク', 'と', '物理的', 'リスク', 'に', 'かかる', 'シナリオ分析', 'の', '高度', '化', 'について', '・', '地域', 'の', 'カーボンニュートラル', '実現', 'に', '向ける', 'た', '各種', '施策', '（', '地域', 'の', 'お客様', 'へ', 'の', 'ソリューション', '支援', '・', '啓発', '策', '、', '地方公共団体', '等', 'と', 'の', '連携', 'を', '含む', '）', '・', '地域', 'の', '取引先', '等', 'に対する', '温室効果ガス', '排出', '量', 'の', '算定', '支援', '（', '温室効果ガス', '排出', '量', '算定', 'サービス', '「', 'しず', 'お', 'か', 'GX', 'サポート', '」', 'の', '展開', '）', '・', '2024年度', 'Sustainable', 'ファイナンス', '(', '環境', '関連', 'ファイナンス', 'を', '含む', ')', '目標', 'の', '設定', '・', 'TNFD', 'Adopter', 'へ', 'の', '登録', 'と', '今後', 'の', '方針', '。', '気候変動', 'による', '当', 'グループ', 'へ', 'の', '影響', 'を', '把握', 'する', 'ため', '、', 'シナリオ分析', '（', '気候変動', 'に関する', 'リスク', 'が', '与信', 'ポートフォリオ', 'に', '与える', '影響', 'を', '把握', '）', 'を', '実施', 'する', 'て', 'いる', 'ます', '。', '使用', 'する', 'た', 'シナリオ', '・', 'IEA', '・', '50年', '実質', 'ゼロ', 'シナリオ', '（',

In [11]:
print(token_noun(text,path1))

['環境委員会', 'カーボンニュートラル', 'Scope', '達成', '2024年度', '以降', '削減', '計画', '移行', 'リスク', '物理的', 'リスク', 'シナリオ分析', '高度', '地域', 'カーボンニュートラル', '実現', '各種', '施策', '地域', 'お客様', 'ソリューション', '支援', '啓発', '地方公共団体', '連携', '地域', '取引先', '温室効果ガス', '排出', '算定', '支援', '温室効果ガス', '排出', '算定', 'サービス', 'しず', 'GX', 'サポート', '展開', '2024年度', 'Sustainable', 'ファイナンス', '環境', '関連', 'ファイナンス', '目標', '設定', 'TNFD', 'Adopter', '登録', '今後', '方針', '気候変動', 'グループ', '影響', '把握', 'シナリオ分析', '気候変動', 'リスク', '与信', 'ポートフォリオ', '影響', '把握', '実施', '使用', 'シナリオ', 'IEA', '50年', '実質', 'シナリオ', 'NZE', 'IEA', 'ETP', 'Beyond', '2度', 'シナリオ', 'NG', 'F・S', 'Net', 'Zero', '分析', '方法', 'シナリオ', '炭素税', '予測', 'データ', '使用', '2050年', '損益', '財政状態', '変化', '予想', '与信', '費用', '変化', '分析', 'm', '使用', 'シナリオ', 'IPCC', '第6次', '報告', 'RCP', '4℃', 'シナリオ']


In [12]:
print(text)
print("--------")
print(token_noun(text,path1))
print("--------")
print(token_noun_with_userdic(text,path1,path2))

環境委員会	・カーボンニュートラル（Scope1、2）達成に向けた2024年度以降の削減計画・移行リスクと物理的リスクにかかるシナリオ分析の高度化について・地域のカーボンニュートラル実現に向けた各種施策（地域のお客さまへのソリューション支援・啓発策、地方公共団体等との連携を含む）・地域の取引先等に対する温室効果ガス排出量の算定支援（温室効果ガス排出量算定サービス「しずおかGXサポート」の展開）・2024年度サステナブルファイナンス(環境関連ファイナンスを含む)目標の設定・TNFD Adopterへの登録と今後の方針。
 気候変動による当グループへの影響を把握するため、シナリオ分析（気候変動に関するリスクが与信ポートフォリオに与える影響を把握）を実施しています。
使用したシナリオ	・IEA・50年実質ゼロシナリオ（NZE2050）・IEA・ETP2017Beyond2℃シナリオ・NGFS Net Zero 2050。
分析方法	シナリオに基づき炭素税等の予測データを使用して、2050年までの損益財政状態の変化を予想し、与信費用の変化を分析。\m使用したシナリオ	IPCC第6次報告書におけるRCP8.5（4℃シナリオ）。
--------
['環境委員会', 'カーボンニュートラル', 'Scope', '達成', '2024年度', '以降', '削減', '計画', '移行', 'リスク', '物理的', 'リスク', 'シナリオ分析', '高度', '地域', 'カーボンニュートラル', '実現', '各種', '施策', '地域', 'お客様', 'ソリューション', '支援', '啓発', '地方公共団体', '連携', '地域', '取引先', '温室効果ガス', '排出', '算定', '支援', '温室効果ガス', '排出', '算定', 'サービス', 'しず', 'GX', 'サポート', '展開', '2024年度', 'Sustainable', 'ファイナンス', '環境', '関連', 'ファイナンス', '目標', '設定', 'TNFD', 'Adopter', '登録', '今後', '方針', '気候変動', 'グループ', '影響', '把握', 'シナリオ分析', '気候変動', 'リスク', '与信', 'ポートフォリオ', '

In [13]:
df.head(20)

,1番目,2番目,count
0,カーボンニュートラル,地域,6
1,2024年度,地域,6
2,リスク,地域,6
3,地域,支援,6
4,地域,温室効果ガス,6
5,地域,排出,6
6,地域,算定,6
7,ファイナンス,地域,6
8,シナリオ,使用,6
9,IEA,シナリオ,6
